<br></br>
# **나이브베이즈 모델의 활용**
1. **베이지안 평가 학습모델**을 불러온다 (0:부정 / 1:긍정)
1. **분석을 위한 네이버 뉴스 댓글**을 수집한다 (600개 내외가 적합)
1. **긍정/ 부정 댓글**을 분류한다
1. **긍정 리뷰**의 **Token** 을 트위터 **Stemming**으로 정규화 한다
1. **정규화된 Token** 중 **"명사", "동사", "형용사"** 만 수집
1. 수집된 Token List 객체를 **nltk.Text 로 변환 후 빈도상위 10개**를 출력한다

<img src="http://www.meconomynews.com/news/photo/201806/15255_14549_2116.png" />

<br></br>
## **1 모듈과 데이터 불러오기**
Import Data / Datum

In [1]:
# Step 1 : 앞에서 학습한 Classifier 모델을 불러온다
# >> classifier = pickle.

import pickle
file = './data/classifier.model'
classifier = pickle.load(open(file, 'rb'))
classifier

In [2]:
# Step 2 : 분류기준 단어목록 불러오기
# >> selected_words = pickle.

file = './data/selected.words'
selected_words = pickle.load(open(file, 'rb'))
selected_words[:5]

['./Punctuation', '하다/Verb', '영화/Noun', '이/Josa', '보다/Verb']

In [3]:
# Step 3 : Tokenizer 생성/ 판단모듈
from konlpy.tag import Twitter
twitter = Twitter()

# 문단을 긍부정 분석용 Token 생성하기
def tokenize(doc):
    doc = ['/'.join(t) for t in twitter.pos(doc, norm=True, stem=True)]
    return {'exists({})'.format(word): (word in set(doc)) for word in selected_words}

# 위에서 생성한 Token을 활용하여 판단하기
def classify_text(text):
    tokens = tokenize(text)             # 트위터로 Stemming/ Taggging 후 Selected_word 판단결과 객체
    return classifier.classify(tokens)  # 분류모델 평가

In [4]:
# Step 4 : 데이터와 모델, 함수를 활용하여 긍/부정 Test
# 0 : 부정모델 , 1 : 긍정모델

text   = """영화가 재미있어요"""
classify_text(text)

'1'

<br></br>
## **2 뉴스 데이터 댓글데이터 불러오기**
replies [참고](http://blog.naver.com/PostView.nhn?blogId=sangdo14&logNo=220610607049&categoryNo=0&parentCategoryNo=163&viewDate=&currentPage=1&postListTopCurrentPage=1&from=search)

In [5]:
from txtutil import Naver_news_rep
news_url = "https://news.naver.com/main/hotissue/read.nhn?mid=hot&" +\
           "sid1=&cid=1064721&iid=3227816&oid=001&aid=0010068060&ptype=021"
replies  = Naver_news_rep(news_url)

import pickle
pickle.dump(replies, open('./data/naver_news.rep', 'wb'))

In [6]:
# 네이버 뉴스 댓글 저장데이터를 
# data 매개변수 이름으로 불러오기

import pickle
data = pickle.load(open('./data/naver_news.rep', 'rb'))
print("Data length : {}\nData sample : {}".format(
    len(data), data[:5]))

Data length : 523
Data sample : ['"방수 된다고 했지, 고장 안난다고 한적 없습나다 고갱님^^"', '"휴지에 물 묻혀서 청소하는게 생활방수 아니면 뭐임??"', '"이러니 순실전자니 뭐니해도 삼성 쓸수밖에.AS가 넘사벽이잖아.왜 소비자가 불편함과 불이익을 감수하면서 까지 불매를 해야함?"', '"실제 애플 홈페이지에 소개된 제품 설명 맨 하단에 보면 작은 글씨로 \\"아이폰은 IP67 등급 생활 방수 및 방진 기능을 갖추고 있다\\"면서도 \\"액체에 의한 손상은 보증 대상이 아니다\\"라고 못 박고 있다. &lt;&lt;&lt; 이부분 어안벙벙하긴 함."', '"침수됐으니 유상으로 수리를 받아야 한다며 76만원을 내라고 했다...이거 실화냐 ? 이게 말이여 ? 방구여 ? ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ"']


<br></br>
## **Mission Test**
긍정 부정댓글 분류하기

In [7]:
# Mission 1
# 전체 댓글 갯수중 긍정/ 부정 갯수 계산하기

In [8]:
pro, con = [], []
for datum in data:
    if classify_text(datum) == "1":
        pro.append(datum)
    else:
        con.append(datum)

In [9]:
result = {}
result["긍정"] = pro
result["부정"] = con

In [10]:
len(result["긍정"])

124

In [11]:
len(result["부정"])

399

In [12]:
# Mission 2
# 긍정 댓글을 구분하여 분류한 뒤
# 명사, 형용사, 동사 Token만 추출하기 (전처리 / Token의 예외처리는 미포함)

In [13]:
%whos

Variable         Type                    Data/Info
--------------------------------------------------
Naver_news_rep   function                <function Naver_news_rep at 0x7f4f4e071ae8>
Twitter          type                    <class 'konlpy.tag._twitter.Twitter'>
classifier       NaiveBayesClassifier    <nltk.classify.naivebayes<...>object at 0x7f4f8461c0f0>
classify_text    function                <function classify_text at 0x7f4f4e33ed90>
con              list                    n=399
data             list                    n=523
datum            str                     "역사가들은 현시대를 친일파강점기라고 부릅니다<...>재인 탄핵과 김대중 부관참시에 동참해주세요."
file             str                     ./data/selected.words
news_url         str                     https://news.naver.com/ma<...>&aid=0010068060&ptype=021
pickle           module                  <module 'pickle' from '/u<...>lib/python3.6/pickle.py'>
pro              list                    n=124
replies          list                    n=523
result     

In [14]:
texts = result['부정']
texts[:3]

['"방수 된다고 했지, 고장 안난다고 한적 없습나다 고갱님^^"',
 '"휴지에 물 묻혀서 청소하는게 생활방수 아니면 뭐임??"',
 '"이러니 순실전자니 뭐니해도 삼성 쓸수밖에.AS가 넘사벽이잖아.왜 소비자가 불편함과 불이익을 감수하면서 까지 불매를 해야함?"']

In [17]:
filtered_tokens = []
for text in texts:
    tokens = twitter.pos(text ,stem=True)
    tokens = [token[0]    for token in tokens    
                          if (token[1] in ['Noun','Adjective','Verb']) and (len(token[0]) > 1 )]
    filtered_tokens += tokens

In [18]:
# Mission 3
# nltk.Text() 객체를 활용하기
# 상위 빈도수 50인 Token을 추출하기

In [19]:
import nltk
pos_nltk = nltk.Text(filtered_tokens, name='긍정댓글')
pos_nltk.vocab().most_common(20)

[('하다', 264),
 ('방수', 178),
 ('되다', 141),
 ('삼성', 121),
 ('아이폰', 97),
 ('쓰다', 96),
 ('없다', 75),
 ('광고', 68),
 ('애플', 63),
 ('아니다', 60),
 ('침수', 54),
 ('닦다', 50),
 ('있다', 49),
 ('받다', 41),
 ('기사', 40),
 ('문제', 38),
 ('기능', 37),
 ('생활', 31),
 ('가다', 30),
 ('수리', 30)]

<br></br>
## **nltk 객체 활용하기**
Token 주변의 단어들 살펴보기

In [20]:
pos_nltk.concordance('조선시대')

no matches
